LaTeX macros (hidden cell)
$
\newcommand{\Q}{\mathcal{Q}}
\newcommand{\ECov}{\boldsymbol{\Sigma}}
\newcommand{\EMean}{\boldsymbol{\mu}}
\newcommand{\EAlpha}{\boldsymbol{\alpha}}
\newcommand{\EBeta}{\boldsymbol{\beta}}
$

# Imports and configuration

In [ ]:
%%bash
FILE=/content/portfolio_tools.py
if [[ ! -f $FILE ]]; then
    wget https://raw.githubusercontent.com/MOSEK/PortfolioOptimization/main/python/notebooks/portfolio_tools.py
fi

In [ ]:
%pip install mosek 
%env PYTHONPATH /env/python:/content
%env MOSEKLM_LICENSE_FILE /content/mosek.lic:/root/mosek/mosek.lic

# To execute the notebook directly in colab make sure your MOSEK license file is in one the locations
#
# /content/mosek.lic   or   /root/mosek/mosek.lic
#
# inside this notebook's internal filesystem. 
#
# You will also need an API key from a stock data provider, or ready data files in a "stock_data" folder.

In [ ]:
import sys
import os
import re
import datetime as dt

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import math
import scipy.stats as stats

from mosek.fusion import *
import mosek.fusion.pythonic          # From MOSEK >= 10.2

from notebook.services.config import ConfigManager

# portfolio_tools.py is a Mosek helper file distributed together with the notebooks
from portfolio_tools import data_download, DataReader, compute_inputs

In [ ]:
# Version checks
print(sys.version)
print('matplotlib: {}'.format(matplotlib.__version__))

# Jupyter configuration
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})  

# Numpy options
np.set_printoptions(precision=5, linewidth=120, suppress=True)

# Pandas options
pd.set_option('display.max_rows', None)

# Matplotlib options
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200

# Prepare input data

Here we load the raw data that will be used to compute the optimization input variables, the vector $\EMean$ of expected returns and the covariance matrix $\ECov$. The data consists of daily stock prices of $8$ stocks from the US market. 

## Download data

In [ ]:
# Data downloading:
# If the user has an API key for alphavantage.co, then this code part will download the data. 
# The code can be modified to download from other sources. To be able to run the examples, 
# and reproduce results in the cookbook, the files have to have the following format and content:
# - File name pattern: "daily_adjusted_[TICKER].csv", where TICKER is the symbol of a stock. 
# - The file contains at least columns "timestamp", "adjusted_close", and "volume".
# - The data is daily price/volume, covering at least the period from 2016-03-18 until 2021-03-18, 
# - Files are for the stocks PM, LMT, MCD, MMM, AAPL, MSFT, TXN, CSCO.
list_stocks = ["PM", "LMT", "MCD", "MMM", "AAPL", "MSFT", "TXN", "CSCO"]
list_factors = []
alphaToken = None
 
list_tickers = list_stocks + list_factors
if alphaToken is not None:
    data_download(list_tickers, alphaToken)  

## Read data

We load the daily stock price data from the downloaded CSV files. The data is adjusted for splits and dividends. Then a selected time period is taken from the data.

In [ ]:
investment_start = "2016-03-18"
investment_end = "2021-03-18"

We split the data into three parts, each having different market regime. The return of each regime will be modeled using a Gaussian distribution.

In [ ]:
# The files are in "stock_data" folder, named as "daily_adjusted_[TICKER].csv"
dr = DataReader(folder_path="stock_data", symbol_list=list_tickers)
dr.read_data()
# Get first regime (low volatility)
df_prices_1, _ = dr.get_period(start_date="2016-03-18", end_date="2018-01-31")
# Get second regime (high volatility: crash)
df_prices_2a, _ = dr.get_period(start_date="2018-02-01", end_date="2018-12-31")
df_prices_2b, _ = dr.get_period(start_date="2020-02-01", end_date="2020-03-23")
# Get third regime (high volatility: recovery)
df_prices_3a, _ = dr.get_period(start_date="2019-01-01", end_date="2020-01-31")
df_prices_3b, _ = dr.get_period(start_date="2020-03-24", end_date="2021-03-18")

# Run the optimization

## Define the optimization model

Below we implement the optimization model in Fusion API. We create it inside a function so we can call it later.

In [ ]:
# log(sum_i(p_i * exp(x_i))) <= t
def logsumexp(M, x, p, t):
    n = int(x.getSize())
    u = M.variable(n)
    M.constraint(Expr.hstack(u, Expr.constTerm(n, 1.0), x - Var.repeat(t, n)), Domain.inPExpCone())
    M.constraint(u.T @ p <= 1.0)

def ExpectedUtility(N, m, G, pi, deltas):

    with Model("Case study") as M:
        # Settings
        #M.setLogHandler(sys.stdout)
        
        assert len(m) == len(G), "Wrong number of components!"
        K = len(m)
        
        # Variables 
        # The variable x is the fraction of holdings relative to the initial capital.
        # It is constrained to take only positive values. 
        x = M.variable("x", N, Domain.greaterThan(0.0))
        
        # Budget constraint
        M.constraint('budget', Expr.sum(x) == 1.0)
        
        # Auxiliary variables.
        z = M.variable("z", 1, Domain.unbounded())
        q = M.variable("q", K, Domain.unbounded())
        
        # Constraint modeling log-sum-exp
        delta = M.parameter()
        mmat = np.vstack(m)
        expconex = delta * (-mmat @ x + delta * q)
        logsumexp(M, expconex, pi, z)
        
        # Constraints modeling quadratic terms
        for i in range(K):
            M.constraint(f'quad_{i}', Expr.vstack(q[i], 1.0, G[i].T @ x), Domain.inRotatedQCone())
        
        # Objective
        M.objective('obj', ObjectiveSense.Minimize, z)
        
        # Create DataFrame to store the results.
        columns = ["delta", "inv_delta", "obj"] + list_stocks
        df_result = pd.DataFrame(columns=columns)
        for d in deltas:
            # Update parameter
            delta.setValue(d);
        
            # Solve optimization
            M.solve()
            
            # Check if the solution is an optimal point
            solsta = M.getPrimalSolutionStatus()
            if (solsta != SolutionStatus.Optimal):
                # See https://docs.mosek.com/latest/pythonfusion/accessing-solution.html about handling solution statuses.
                raise Exception("Unexpected solution status!")

            # Save results
            row = pd.Series([d, 1/d, M.primalObjValue()] + list(x.level()), index=columns)
            df_result = pd.concat([df_result, pd.DataFrame([row])], ignore_index=True)
            
        return df_result

## Compute optimization input variables

Here we use the loaded daily price data to compute the corresponding yearly mean return and covariance matrix.

In [ ]:
# Regime probabilities
pi1 = 0.6
pi2 = 0.1
pi3 = 0.3

# Get optimization parameters
m1, S1 = compute_inputs([df_prices_1])
m2, S2 = compute_inputs([df_prices_2a, df_prices_2b])
m3, S3 = compute_inputs([df_prices_3a, df_prices_3b])

# Number of securities
N = m1.shape[0]

Next we compute the matrix $G$ such that $\ECov=GG^\mathsf{T}$, this is the input of the conic form of the optimization problem. Here we use Cholesky factorization.

In [ ]:
G1 = np.linalg.cholesky(S1)
G2 = np.linalg.cholesky(S2)
G3 = np.linalg.cholesky(S3)

## Call the optimizer function

We run the optimization for a range of risk aversion parameter values: $\delta = 10^{-1},\dots,10^{2}$. We compute the efficient frontier this way both with and without using shrinkage estimation. 

In [ ]:
# Compute efficient frontier with and without shrinkage
deltas = np.logspace(start=-0.5, stop=3, num=30)[::-1]
df_result = ExpectedUtility(N, [m1, m2, m3], [G1, G2, G3], [pi1, pi2, pi3], deltas)

Check the results.

In [ ]:
df_result

## Visualize the results

Plot the portfolio composition.

In [ ]:
# Round small values to 0 to make plotting work
mask = np.absolute(df_result) < 1e-7
mask.iloc[:, :-8] = False
df_result[mask] = 0

In [ ]:
my_cmap = LinearSegmentedColormap.from_list("non-extreme gray", ["#111111", "#eeeeee"], N=256, gamma=1.0)
ax = df_result.set_index('inv_delta').iloc[:, 2:].plot.area(colormap=my_cmap, xlabel='risk seeking', ylabel="x", logx=True) 
ax.grid(which='both', axis='x', linestyle=':', color='k', linewidth=1)